In [1]:
from fastcore.utils import *
from ghapi import *
import tarfile,pprint

In [5]:
tag = 'v0.79.0'
api = GhApi(owner='fastai', repo='hugo-mathjax', token=github_token())
rel = api.repos.get_release_by_tag(tag)
api.delete_release(rel)

In [2]:
from fastcore.utils import *
from ghapi import *
import tarfile

def rel_tag():
    api = GhApi(owner='fastai', repo='hugo-mathjax', token=github_token())
    tag = GhApi().repos.get_latest_release('gohugoio', repo='hugo').name
    try: return api.repos.get_release_by_tag(tag)
    except HTTP404NotFoundError: pass

    rel = api.repos.create_release(tag, name=tag)
    actions_output('tag', tag)

# rel_tag()

In [14]:
github_needs = urljson('https://raw.githubusercontent.com/fastai/ghapi/master/examples/needs.json?')
out = loads(nested_idx(github_needs, 'prebuild', 'outputs', 'out')); out

{'step1': {'outputs': {'action_fruit': 'strawberry'},
  'outcome': 'success',
  'conclusion': 'success'}}

In [5]:
tag = nested_idx(out, 'step1', 'outputs', 'tag'); tag

In [59]:
# os.chdir('..')

platform = dict(linux='linux', linux2='linux', win32='win', darwin='mac')[sys.platform]
out = loads(nested_idx(github_needs, 'prebuild', 'outputs', 'out'))
tag = nested_idx(out, 'step1', 'outputs', 'tag')

rel = GhApi().repos.get_release_by_tag(owner='gohugoio', repo='hugo', tag=tag)
print(rel.tarball_url)
with urlopen(rel.tarball_url) as f: untar_dir(f, repo)
os.chdir('hugo')
run(f'patch -l -p1 -i ../hugo.patch')
run('go build')# --tags extended')
ext_nm = 'hugo.exe' if platform=='win' else 'hugo'
fn = f'hugo-{platform}.tgz'
with tarfile.open(fn, "w:gz") as tar: tar.add(ext_nm)

api = GhApi(owner='fastai', repo='hugo-mathjax', token=github_token())
rel = api.repos.get_release_by_tag(tag)
api.upload_file(rel, fn)